#데이터준비

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq "/content/drive/MyDrive/ColabNotebooks/빅최기/hw3/KCC150_Korean_sentences_EUCKR.zip"

In [ ]:
!unzip -qq "/content/drive/MyDrive/ColabNotebooks/빅최기/hw3/triFreq-Sejong_utf8.zip"

In [ ]:
# -*- coding: utf-8 -*-
fe = open("/content/triFreq-Sejong_utf8.txt",encoding='utf-8')
linee = fe.readlines()

print(linee[:10])

['     1\t\x01_S\n', '     1\t\x07마이\n', '    58\t!!$\n', '     2\t!!(\n', '     3\t!!)\n', '     2\t!!/\n', '     3\t!!>\n', '     5\t!!]\n', '   110\t!!_\n', '     2\t!!…\n']


빈도수가 10 이상이고, 한글만 포함하는 텍스트만 추출

In [ ]:
import sys
import re

# def isHangul(text):
#     pyVer3 =  sys.version_info >= (3, 0)

#     if pyVer3 : # for Ver 3 or later
#         encText = text
#     else: # for Ver 2.x
#         if type(text) is not unicode:
#             encText = text.decode('utf-8')
#         else:
#             encText = text

#     hanCount = len(re.findall(u'[\u3130-\u318F\uAC00-\uD7A3]+', encText))
#     return hanCount > 0

# #print(han + " is " + str(isHangul(han)))

In [ ]:
def isEnglishOrKorean(input_s):
    k_count = 0
    e_count = 0
    for c in input_s:
        if ord('가') <= ord(c) <= ord('힣'):
            k_count+=1
        elif ord('a') <= ord(c.lower()) <= ord('z'):
            e_count+=1
    return "k" if k_count>2 else "e"

In [ ]:
vecdic = dict()
cntdic = []
worddic = []
cnt = 0
for a in linee :
  a = a.replace(" ", "")
  a = a.replace("  ", "")
  a = a.replace("   ", "")
  a = a.replace("    ", "")
  a = a.replace("\t", "-")
  a = a.replace("\n", "")
  a = a.split('-')
  if int(a[0]) > 10 and str(isEnglishOrKorean(a[1]))=='k':
    worddic.append(a[1])
  

N = 751021 ( N -- 10만~100만 내외 )

In [ ]:
print(len(worddic))

359329


In [ ]:
worddic.sort()

In [ ]:
worddic[:10]

['가가게', '가가고', '가가기', '가가는', '가가니', '가가대', '가가더', '가가던', '가가도', '가가려']

각 trigram에 대해 정수 id를 1,2,3,...,N으로 부여

In [ ]:
worddicidx = {cls:idx for idx, cls in enumerate(worddic, start=1)}
worddicidx

{'가가게': 1,
 '가가고': 2,
 '가가기': 3,
 '가가는': 4,
 '가가니': 5,
 '가가대': 6,
 '가가더': 7,
 '가가던': 8,
 '가가도': 9,
 '가가려': 10,
 '가가린': 11,
 '가가며': 12,
 '가가면': 13,
 '가가서': 14,
 '가가야': 15,
 '가가자': 16,
 '가가지': 17,
 '가가치': 18,
 '가가호': 19,
 '가간다': 20,
 '가간에': 21,
 '가간의': 22,
 '가갈수': 23,
 '가감없': 24,
 '가감하': 25,
 '가갔고': 26,
 '가갔다': 27,
 '가갔던': 28,
 '가갔습': 29,
 '가갔을': 30,
 '가갔지': 31,
 '가강습': 32,
 '가강의': 33,
 '가개념': 34,
 '가개발': 35,
 '가개방': 36,
 '가개입': 37,
 '가개혁': 38,
 '가객들': 39,
 '가객이': 40,
 '가갸거': 41,
 '가거나': 42,
 '가거도': 43,
 '가거든': 44,
 '가거들': 45,
 '가거라': 46,
 '가건만': 47,
 '가건물': 48,
 '가건설': 49,
 '가검물': 50,
 '가겄나': 51,
 '가겄다': 52,
 '가겄소': 53,
 '가게가': 54,
 '가게까': 55,
 '가게끔': 56,
 '가게나': 57,
 '가게는': 58,
 '가게다': 59,
 '가게도': 60,
 '가게되': 61,
 '가게된': 62,
 '가게들': 63,
 '가게라': 64,
 '가게로': 65,
 '가게를': 66,
 '가게마': 67,
 '가게만': 68,
 '가게무': 69,
 '가게문': 70,
 '가게보': 71,
 '가게세': 72,
 '가게에': 73,
 '가게였': 74,
 '가게와': 75,
 '가게의': 76,
 '가게이': 77,
 '가게인': 78,
 '가게일': 79,
 '가게주': 80,
 '가게집': 81,
 '가게터': 82,
 '가겟방': 83,
 '가겟집': 84,
 

# kcc150파일에 대한 벡터만들기

In [ ]:
# -*- coding: cp949 -*-
f = open("/content/KCC150_Korean_sentences_EUCKR.txt", encoding='CP949')
lines = f.readlines()
print(len(lines))


11961347


* wordcount_list -> 문장과 빈도벡터(word_count의 value값)가 포함되어있음 ( 문장 : [0,0,0,0,0,0,1,0,0,0...] )
* word_count -> 해당 문장의 trigram 키값과 tf값 ( trigram1 : 0 , trigram2 : 0 ....)
* 문장 1000개 사용



In [ ]:
a=0
wordcount_list = {}
wordList = worddic

for line in lines[:1000] :
  word_Count = {} 

  for word in wordList:
    word_Count[worddicidx[word]] = word_Count.get(worddicidx[word], 0)
    if word in line :
      word_Count[worddicidx[word]] = word_Count.get(worddicidx[word]) + 1 
  wordcount_list[a] = list(word_Count.values())
  a = a+1
  print(a)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [ ]:
wordcount_list

유사도계산 함수

In [ ]:
from numpy import dot
from numpy.linalg import norm
import numpy as np
def cos_sim(A, B):
       return dot(A, B)/(norm(A)*norm(B))

* 1000개의 문장에 대한 벡터가 담겨있는 gettop
* 문장의 number를 인자로 주면 유사도가 가장높은 상위3개의 문장을 추출해주는 함수 getsimilarity()

In [ ]:
print(type(wordcount_list.get(1)))

<class 'list'>


In [ ]:
b = [0,1,0,0,0]
any(b)

True

In [ ]:
def getsimilarity(n):
  gettop = {}
  for b in range(0,len(wordcount_list)):
    if b == n : continue
    else :
      if any(wordcount_list.get(n)) == True and any(wordcount_list.get(b)) == True :
        gettop['[%s : %s]' %(lines[n],lines[b])] = cos_sim(wordcount_list.get(n),wordcount_list.get(b))
      # gettop['%d : %d [%s : %s]' %(n,b,lines[n],lines[b])] = cos_sim(wordcount_list.get(lines[n]),wordcount_list.get(lines[b]))
  # print(gettop)
  gettop = dict(sorted(gettop.items(), key=lambda x: x[1], reverse=True))
  keey = list(gettop.keys())
  a = 0
  for k in keey :
    if a < 3 :
      print(k, gettop[k])
      a = a+1
    else : break
  return
    

In [ ]:
getsimilarity(1)

[이에 따라 전달된 후원금은 저소득층과 사회복지시설에 2억원 상당을, 중구 푸드뱅크 설립 지원에 6000만원, 저소득 긴급지원과 시설 등의 지원에 5000만원이 전해진다.
 : 충북본부는 장보기를 통해 구입한 물건을 인근 사회복지시설에 기부했다.
] 0.3227486121839514
[이에 따라 전달된 후원금은 저소득층과 사회복지시설에 2억원 상당을, 중구 푸드뱅크 설립 지원에 6000만원, 저소득 긴급지원과 시설 등의 지원에 5000만원이 전해진다.
 : 올해로 4회째를 맞는 이번 행사는 까리따스 방배종합사회복지관과 함께 진행하는 지역사회 나눔 실천 활동으로, 이날은 방배동에 거주하는 어린이 및 주민 400여명과 CJ오쇼핑 임직원 30여명이 참여했다.
] 0.08770580193070293
[이에 따라 전달된 후원금은 저소득층과 사회복지시설에 2억원 상당을, 중구 푸드뱅크 설립 지원에 6000만원, 저소득 긴급지원과 시설 등의 지원에 5000만원이 전해진다.
 : 전달된 수익금은 해당 지역아동센터의 학습 지도 운영비로 사용될 예정이다.
] 0.07071067811865474


In [ ]:
getsimilarity(33)

[마리오 드라기 유럽중앙은행 총재가 또 초대형 경기 부양에 나선다.
 : G20 재무장관과 중앙은행 총재들은 각국의 구조개혁에도 주목했다.
] 0.1781741612749496
[마리오 드라기 유럽중앙은행 총재가 또 초대형 경기 부양에 나선다.
 : 공연은 말없이 다가가는 세련된 말 걸기 콘셉트로 마네킹 일루션, 클래식과 마임 현대무용의 콜라보 공연, 마임광대의 서커스 공연, 버블과 마리오네트 공연 등 다양한 프로그램이 진행된다.
] 0.06950480468569159
[마리오 드라기 유럽중앙은행 총재가 또 초대형 경기 부양에 나선다.
 : 한국도로공사는 2010년 4월1일부터 사용이 중지된 고속도로 카드의 잔액을 국민들이 제때 환불받을 수 있도록 오는 16일부터 6월말까지 집중 홍보기간으로 정하고 대국민 홍보에 나선다고 밝혔다.
] 0.061898446059017294


In [ ]:
getsimilarity(423)

In [ ]:
getsimilarity(666)